# **Neuroimaging 2025/2026**

**Project 10: Multimodal Age Group Classification**

Tutors: Beatriz Vale & Vânia Miguel

Created by: [Your Names Here]

---

> **Dataset:** MPI-Leipzig Mind-Brain-Body (LEMON)
>
> **Modalities:** MRI + DWI + fMRI + EEG
>
> **Subjects:** 10 (5 young, 5 old)
>
> **Goal:** Extract features from each modality using techniques from Labs 1-5, train a simple classifier, and compare which modalities are most informative.

---

⚠️ **Important:** With only 10 subjects, we do NOT expect high classification accuracy. The focus of this project is on the **process**: correctly applying each pipeline, extracting meaningful features, and critically discussing the results and limitations.

## **Objectives**
```
* Apply structural MRI processing (Lab 1) to extract subcortical volumes with FIRST
* Apply diffusion MRI processing (Lab 2) to extract FA from white matter tracts
* Apply fMRI connectivity analysis (Lab 4) to extract functional connectivity features
* Apply EEG spectral analysis (Lab 5) to extract band power and alpha peak frequency
* Integrate multimodal features and train a simple classifier (young vs old)
* Compare unimodal vs multimodal classification performance
* Critically discuss results and limitations
```

# **I. Set up the environment**

We need both Neurodesk (for FSL and MRtrix3) and MNE-Python (for EEG). We also need scikit-learn for the classification step.

---

In [ ]:
# Set up Neurodesk
%%capture
import os
import sys
IN_COLAB = 'google.colab' in sys.modules

if IN_COLAB:
    os.environ["LD_PRELOAD"] = "";
    os.environ["APPTAINER_BINDPATH"] = "/content,/tmp,/cvmfs"
    os.environ["MPLCONFIGDIR"] = "/content/matplotlib-mpldir"
    os.environ["LMOD_CMD"] = "/usr/share/lmod/lmod/libexec/lmod"

    !curl -J -O https://raw.githubusercontent.com/NeuroDesk/neurocommand/main/googlecolab_setup.sh
    !chmod +x googlecolab_setup.sh
    !./googlecolab_setup.sh

    os.environ["MODULEPATH"] = ':'.join(map(str, list(map(lambda x: os.path.join(os.path.abspath('/cvmfs/neurodesk.ardc.edu.au/neurodesk-modules/'), x), os.listdir('/cvmfs/neurodesk.ardc.edu.au/neurodesk-modules/')))))

In [ ]:
# Load neuroimaging tools
import lmod
await lmod.load('fsl/6.0.7.16')
await lmod.load('mrtrix3/3.0.4')

In [ ]:
# Install Python packages for EEG and ML
!pip3 install -q mne==1.9.0 mne_icalabel==0.7.0 scikit-learn pandas seaborn pymatreader

In [ ]:
# Core imports
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

from IPython.display import display, Markdown

# **II. Download LEMON data**

We select 10 subjects: 5 young (20-35y) and 5 old (59-77y), all with T1w, DWI, fMRI, and EEG available.

**Data sources:**
- **MRI / fMRI / DWI:** OpenNeuro ds000221 (ses-01)
- **EEG:** GWDG FTP or NITRC-INDI (separate download)
- **ID mapping:** CSV file to match INDI IDs ↔ original LEMON IDs

---

### 1. Select subjects

Before downloading, you need to identify subjects that have **all 4 modalities**. Use the data availability CSV from the LEMON dataset page to find matching subjects.

⚠️ The EEG data uses different subject IDs than OpenNeuro. Use the ID mapping CSV from the INDI page to match them.

---

In [ ]:
# Define our 10 subjects (replace with your actual subject IDs after checking availability)
# These are example IDs from OpenNeuro ds000221 (ses-01)
young_subjects = ['sub-010002', 'sub-010003', 'sub-010004', 'sub-010005', 'sub-010006']
old_subjects   = ['sub-010017', 'sub-010018', 'sub-010019', 'sub-010020', 'sub-010021']
all_subjects   = young_subjects + old_subjects
labels         = [0]*5 + [1]*5  # 0 = young, 1 = old
label_names    = ['Young']*5 + ['Old']*5

print(f"Total subjects: {len(all_subjects)}")
print(f"Young: {len(young_subjects)}, Old: {len(old_subjects)}")

### 2. Download MRI / fMRI / DWI from OpenNeuro

Using DataLad to download only the specific subjects and sessions we need.

---

In [ ]:
# Install DataLad and clone the dataset (metadata only, no data yet)
!pip install -q datalad
!git config --global user.name "Student"
!git config --global user.email "student@example.com"

# Clone the dataset (this only downloads metadata, ~fast)
!datalad install https://github.com/OpenNeuroDatasets/ds000221.git
os.chdir('ds000221')

In [ ]:
# Download data for each subject (ses-01 only: LEMON protocol)
# This downloads T1w, fMRI, and DWI for each subject
for sub in all_subjects:
    print(f"\n--- Downloading {sub} ---")
    !datalad get {sub}/ses-01/anat/ -J 4
    !datalad get {sub}/ses-01/func/ -J 4
    !datalad get {sub}/ses-01/dwi/ -J 4

### 3. Download EEG data

EEG data is hosted separately. Download from the LEMON EEG page.

---

In [ ]:
# Download EEG data for matching subjects
# Replace URLs with the actual LEMON EEG download links for your subjects
# The EEG data is organized per subject as .tar.gz files

EEG_BASE_URL = "https://ftp.gwdg.de/pub/misc/MPI-Leipzig_Mind-Brain-Body-LEMON/EEG_MPILMBB_LEMON/EEG_Raw_BIDS_ID/"

os.makedirs('/content/eeg_data', exist_ok=True)

# NOTE: You need to map OpenNeuro IDs to LEMON EEG IDs using the CSV file
# Example (replace with your actual mapped IDs):
eeg_ids = {
    'sub-010002': 'sub-010002',  # Replace with actual EEG ID mapping
    # ... add all 10 subjects
}

# Download EEG for each subject (adapt URL pattern to actual structure)
for sub, eeg_id in eeg_ids.items():
    print(f"Downloading EEG for {sub} ({eeg_id})...")
    # !wget -q -P /content/eeg_data/ {EEG_BASE_URL}/{eeg_id}.tar.gz
    # !tar xzf /content/eeg_data/{eeg_id}.tar.gz -C /content/eeg_data/
    
print("⚠️ Adapt the download commands above to the actual EEG file structure.")

### 4. Verify data availability

Check that all expected files exist for each subject before proceeding.

---

In [ ]:
# Verify data files exist for each subject
print("=" * 70)
print(f"{'Subject':<15} {'T1w':>5} {'fMRI':>5} {'DWI':>5} {'EEG':>5}")
print("=" * 70)

for sub in all_subjects:
    has_t1  = os.path.exists(f'{sub}/ses-01/anat/')
    has_func = os.path.exists(f'{sub}/ses-01/func/')
    has_dwi = os.path.exists(f'{sub}/ses-01/dwi/')
    has_eeg = os.path.exists(f'/content/eeg_data/{sub}/')
    
    status = lambda x: '  ✓' if x else '  ✗'
    print(f"{sub:<15} {status(has_t1):>5} {status(has_func):>5} {status(has_dwi):>5} {status(has_eeg):>5}")

# **III. MRI Structural Features (Lab 1)**

We extract **subcortical volumes** using FIRST (as learned in Lab 1).

**Pipeline:** Brain extraction (BET) → Subcortical segmentation (FIRST) → Volume extraction (fslstats)

**Features extracted per subject:** Volume of L/R thalamus, hippocampus, caudate, putamen, pallidum, amygdala, accumbens (14 values).

---

### 1. Brain extraction and FIRST segmentation

Apply BET and run_first_all for each subject, exactly as in Lab 1.

---

In [ ]:
# Process each subject: BET + FIRST
mri_features = {}

for sub in all_subjects:
    print(f"\n{'='*50}")
    print(f"Processing {sub} - Structural MRI")
    print(f"{'='*50}")
    
    anat_dir = f'{sub}/ses-01/anat/'
    # Find T1w file (naming may vary)
    t1_file = !ls {anat_dir}/*T1w.nii.gz 2>/dev/null | head -1
    t1_file = t1_file[0] if t1_file else None
    
    if not t1_file:
        print(f"  ⚠️ No T1w found for {sub}, skipping.")
        continue
    
    print(f"  T1w file: {t1_file}")
    
    # Create working directory
    work_dir = f'/content/mri_work/{sub}'
    os.makedirs(work_dir, exist_ok=True)
    
    # Step 1: Brain extraction with BET (Lab 1)
    print("  Step 1: Brain extraction (BET)...")
    !bet {t1_file} {work_dir}/T1_brain -R -f 0.5
    
    # Step 2: FIRST subcortical segmentation (Lab 1)
    print("  Step 2: Subcortical segmentation (FIRST)...")
    !run_first_all -i {work_dir}/T1_brain -b -o {work_dir}/T1_first
    
    print(f"  ✓ Done: {sub}")

### 2. Extract subcortical volumes

Use fslstats to extract the volume (in mm³) of each subcortical structure, as in Lab 1.

---

In [ ]:
# Subcortical structures segmented by FIRST
structures = [
    'L_Thal', 'R_Thal', 'L_Hipp', 'R_Hipp',
    'L_Caud', 'R_Caud', 'L_Puta', 'R_Puta',
    'L_Pall', 'R_Pall', 'L_Amyg', 'R_Amyg',
    'L_Accu', 'R_Accu'
]

mri_features = {}

for sub in all_subjects:
    work_dir = f'/content/mri_work/{sub}'
    seg_file = f'{work_dir}/T1_first_all_fast_firstseg.nii.gz'
    
    if not os.path.exists(seg_file):
        print(f"⚠️ FIRST output missing for {sub}")
        mri_features[sub] = [np.nan] * len(structures)
        continue
    
    volumes = []
    for i, struct in enumerate(structures):
        # FIRST labels are indexed starting from specific values
        # Extract volume of non-zero voxels for each structure mesh
        mesh_file = f'{work_dir}/T1_first-{struct}_first.nii.gz'
        if os.path.exists(mesh_file):
            result = !fslstats {mesh_file} -V
            vol_mm3 = float(result[0].split()[1]) if result else np.nan
        else:
            vol_mm3 = np.nan
        volumes.append(vol_mm3)
    
    mri_features[sub] = volumes
    print(f"{sub}: extracted {len([v for v in volumes if not np.isnan(v)])} structure volumes")

# Create DataFrame
df_mri = pd.DataFrame(mri_features, index=structures).T
df_mri.index.name = 'Subject'
print("\nMRI Feature Matrix:")
df_mri

### 3. QC: Visualize one example

Always check results visually!

---

In [ ]:
# Visualize FIRST segmentation for one subject
example_sub = all_subjects[0]
work_dir = f'/content/mri_work/{example_sub}'

from nilearn import plotting

# Plot subcortical segmentation overlaid on T1
plotting.plot_roi(
    f'{work_dir}/T1_first_all_fast_firstseg.nii.gz',
    bg_img=f'{work_dir}/T1_brain.nii.gz',
    title=f'FIRST Segmentation - {example_sub}',
    display_mode='ortho',
    cut_coords=(0, -20, 10)
)
plt.show()

# **IV. Diffusion MRI Features (Lab 2)**

We extract **FA (Fractional Anisotropy)** values from major white matter tracts using the JHU atlas.

**Pipeline:** Denoising (dwidenoise) → Preprocessing (dwifslpreproc) → Tensor fitting (dwi2tensor + tensor2metric) → FA extraction from JHU ROIs

**Features extracted per subject:** Mean FA in 5 major tracts (genu CC, body CC, splenium CC, SLF, corticospinal tract) = 5 values.

---

### 1. DWI preprocessing and tensor fitting

Apply the preprocessing pipeline from Lab 2 to each subject.

---

In [ ]:
# Process each subject: DWI preprocessing + tensor fitting
dwi_features = {}

for sub in all_subjects:
    print(f"\n{'='*50}")
    print(f"Processing {sub} - Diffusion MRI")
    print(f"{'='*50}")
    
    dwi_dir = f'{sub}/ses-01/dwi/'
    dwi_file = !ls {dwi_dir}/*dwi.nii.gz 2>/dev/null | head -1
    dwi_file = dwi_file[0] if dwi_file else None
    
    if not dwi_file:
        print(f"  ⚠️ No DWI found for {sub}, skipping.")
        continue
    
    bval_file = dwi_file.replace('.nii.gz', '.bval')
    bvec_file = dwi_file.replace('.nii.gz', '.bvec')
    
    work_dir = f'/content/dwi_work/{sub}'
    os.makedirs(work_dir, exist_ok=True)
    
    # Step 1: Denoising (Lab 2)
    print("  Step 1: Denoising (dwidenoise)...")
    !dwidenoise {dwi_file} {work_dir}/dwi_den.mif -fslgrad {bvec_file} {bval_file} -force
    
    # Step 2: Preprocessing (Lab 2) 
    # Note: simplified version without fieldmaps
    print("  Step 2: Preprocessing (dwifslpreproc)...")
    !dwifslpreproc {work_dir}/dwi_den.mif {work_dir}/dwi_preproc.mif -rpe_none -pe_dir AP -force
    
    # Step 3: Brain mask
    print("  Step 3: Brain mask (dwi2mask)...")
    !dwi2mask {work_dir}/dwi_preproc.mif {work_dir}/mask.mif -force
    
    # Step 4: Tensor fitting and FA/MD maps (Lab 2)
    print("  Step 4: Tensor fitting (dwi2tensor)...")
    !dwi2tensor {work_dir}/dwi_preproc.mif {work_dir}/dt.mif -mask {work_dir}/mask.mif -force
    !tensor2metric {work_dir}/dt.mif -fa {work_dir}/fa.mif -mask {work_dir}/mask.mif -force
    
    # Convert to NIfTI for fslstats
    !mrconvert {work_dir}/fa.mif {work_dir}/fa.nii.gz -force
    
    print(f"  ✓ Done: {sub}")

### 2. Extract FA from JHU white matter atlas ROIs

Use the JHU atlas from FSL to extract mean FA in major tracts, using fslstats as in Lab 2.

---

In [ ]:
# JHU White Matter Atlas tract labels (selected major tracts)
# These correspond to specific label indices in the JHU-ICBM atlas
jhu_tracts = {
    'Genu_CC': 3,        # Genu of corpus callosum
    'Body_CC': 4,        # Body of corpus callosum  
    'Splenium_CC': 5,    # Splenium of corpus callosum
    'CST_R': 8,          # Corticospinal tract R
    'SLF_R': 20,         # Superior longitudinal fasciculus R
}

# JHU atlas location in FSL
jhu_atlas = os.path.join(os.environ.get('FSLDIR', ''), 
                         'data/atlases/JHU/JHU-ICBM-labels-1mm.nii.gz')

print(f"JHU Atlas: {jhu_atlas}")
print(f"Exists: {os.path.exists(jhu_atlas)}")

dwi_features = {}

for sub in all_subjects:
    work_dir = f'/content/dwi_work/{sub}'
    fa_file = f'{work_dir}/fa.nii.gz'
    
    if not os.path.exists(fa_file):
        print(f"⚠️ FA map missing for {sub}")
        dwi_features[sub] = [np.nan] * len(jhu_tracts)
        continue
    
    # Register FA to MNI space for atlas overlap
    # Using FLIRT (linear registration, as in Lab 1)
    mni_template = os.path.join(os.environ.get('FSLDIR', ''),
                                'data/standard/MNI152_T1_1mm_brain.nii.gz')
    
    !flirt -in {fa_file} -ref {mni_template} -out {work_dir}/fa_mni.nii.gz \
           -omat {work_dir}/fa2mni.mat -dof 12
    
    fa_vals = []
    for tract_name, label_idx in jhu_tracts.items():
        # Create tract mask from JHU atlas
        !fslmaths {jhu_atlas} -thr {label_idx} -uthr {label_idx} -bin {work_dir}/tract_mask.nii.gz
        
        # Extract mean FA within tract
        result = !fslstats {work_dir}/fa_mni.nii.gz -k {work_dir}/tract_mask.nii.gz -M
        mean_fa = float(result[0]) if result else np.nan
        fa_vals.append(mean_fa)
    
    dwi_features[sub] = fa_vals
    print(f"{sub}: FA extracted from {len(jhu_tracts)} tracts")

# Create DataFrame
df_dwi = pd.DataFrame(dwi_features, index=list(jhu_tracts.keys())).T
df_dwi.index.name = 'Subject'
print("\nDWI Feature Matrix:")
df_dwi

### 3. QC: Visualize FA map for one subject

---

In [ ]:
# Visualize FA map for one subject
example_sub = all_subjects[0]
work_dir = f'/content/dwi_work/{example_sub}'

from nilearn import plotting

plotting.plot_anat(
    f'{work_dir}/fa.nii.gz',
    title=f'FA Map - {example_sub}',
    display_mode='ortho',
    cmap='hot',
    vmin=0, vmax=1
)
plt.show()

# **V. Functional Connectivity Features (Lab 4)**

We extract **functional connectivity** features from resting-state fMRI using the Schaefer atlas.

**Pipeline:** Basic preprocessing (motion correction + smoothing) → Parcellation (Schaefer 100) → FC matrix → Network-level summary features

**Features extracted per subject:** Mean intra-network and inter-network connectivity for 7 Yeo networks = ~28 values (7 intra + 21 inter).

---

### 1. Basic fMRI preprocessing

Apply minimal preprocessing: motion correction and smoothing, as in Lab 3 Part 1.

---

In [ ]:
# Process each subject: basic fMRI preprocessing
for sub in all_subjects:
    print(f"\n{'='*50}")
    print(f"Processing {sub} - fMRI")
    print(f"{'='*50}")
    
    func_dir = f'{sub}/ses-01/func/'
    func_file = !ls {func_dir}/*bold.nii.gz 2>/dev/null | head -1
    func_file = func_file[0] if func_file else None
    
    if not func_file:
        print(f"  ⚠️ No fMRI found for {sub}, skipping.")
        continue
    
    work_dir = f'/content/fmri_work/{sub}'
    os.makedirs(work_dir, exist_ok=True)
    
    # Check dimensions
    !fslinfo {func_file} | grep -E "^dim|^pixdim"
    
    # Step 1: Motion correction (Lab 3)
    print("  Step 1: Motion correction (mcflirt)...")
    !mcflirt -in {func_file} -out {work_dir}/func_mc -plots
    
    # Step 2: Brain extraction on mean functional
    print("  Step 2: Skull stripping (bet)...")
    !fslmaths {work_dir}/func_mc -Tmean {work_dir}/func_mean
    !bet {work_dir}/func_mean {work_dir}/func_brain -f 0.3 -m
    
    # Step 3: Spatial smoothing (Lab 3, FWHM=6mm -> sigma=2.548)
    print("  Step 3: Smoothing (fslmaths -s)...")
    !fslmaths {work_dir}/func_mc -s 2.548 {work_dir}/func_smooth
    
    # Step 4: Temporal filtering (bandpass 0.01-0.1 Hz)
    # Get TR for this subject
    tr_result = !fslval {func_file} pixdim4
    tr = float(tr_result[0]) if tr_result else 2.0
    hp_sigma = int(1.0 / (2 * 0.01 * tr))  # High-pass sigma in volumes
    lp_sigma = int(1.0 / (2 * 0.1 * tr))   # Low-pass sigma in volumes
    
    print(f"  Step 4: Bandpass filtering (TR={tr}s, hp_sigma={hp_sigma}, lp_sigma={lp_sigma})...")
    !fslmaths {work_dir}/func_smooth -bptf {hp_sigma} {lp_sigma} {work_dir}/func_filtered
    
    print(f"  ✓ Done: {sub}")

### 2. Extract functional connectivity features

Parcellate with Schaefer 100 atlas and compute FC matrix, exactly as in Lab 4.

---

In [ ]:
# Extract FC features using nilearn (same approach as Lab 4)
from nilearn import datasets, maskers, connectome

# Load Schaefer atlas (same as Lab 4)
atlas = datasets.fetch_atlas_schaefer_2018(n_rois=100, yeo_networks=7)
atlas_filename = atlas.maps
labels = [l.decode() if isinstance(l, bytes) else l for l in atlas.labels[1:]]

# Define the 7 Yeo networks
network_names = ['Vis', 'SomMot', 'DorsAttn', 'SalVentAttn', 'Limbic', 'Cont', 'Default']
network_ids = []
for label in labels:
    for i, net in enumerate(network_names):
        if net in label:
            network_ids.append(i)
            break

network_ids = np.array(network_ids)
print(f"Network assignment: {len(network_ids)} regions across {len(network_names)} networks")

In [ ]:
# Compute FC matrix for each subject and extract network-level features
fmri_features = {}

for sub in all_subjects:
    work_dir = f'/content/fmri_work/{sub}'
    func_file = f'{work_dir}/func_filtered.nii.gz'
    
    if not os.path.exists(func_file):
        print(f"⚠️ Filtered fMRI missing for {sub}")
        fmri_features[sub] = [np.nan] * 28
        continue
    
    # Parcellate with Schaefer 100 (same as Lab 4)
    masker = maskers.NiftiLabelsMasker(
        labels_img=atlas_filename,
        standardize='zscore_sample',
        memory='nilearn_cache',
        verbose=0
    )
    
    try:
        time_series = masker.fit_transform(func_file)
        
        # Compute FC matrix (Pearson correlation, same as Lab 4)
        correlation_measure = connectome.ConnectivityMeasure(kind='correlation')
        fc_matrix = correlation_measure.fit_transform([time_series])[0]
        
        # Extract network-level features: mean intra- and inter-network FC
        n_networks = len(network_names)
        features = []
        feature_names_fc = []
        
        # Intra-network connectivity (7 values)
        for i in range(n_networks):
            mask = network_ids == i
            intra_vals = fc_matrix[np.ix_(mask, mask)]
            # Upper triangle only (exclude diagonal)
            triu = intra_vals[np.triu_indices(intra_vals.shape[0], k=1)]
            features.append(np.mean(triu) if len(triu) > 0 else np.nan)
            feature_names_fc.append(f'Intra_{network_names[i]}')
        
        # Inter-network connectivity (21 values)
        for i in range(n_networks):
            for j in range(i+1, n_networks):
                mask_i = network_ids == i
                mask_j = network_ids == j
                inter_vals = fc_matrix[np.ix_(mask_i, mask_j)]
                features.append(np.mean(inter_vals))
                feature_names_fc.append(f'Inter_{network_names[i]}_{network_names[j]}')
        
        fmri_features[sub] = features
        print(f"{sub}: {len(features)} FC features extracted (FC matrix shape: {fc_matrix.shape})")
        
    except Exception as e:
        print(f"⚠️ Error processing {sub}: {e}")
        fmri_features[sub] = [np.nan] * 28

# Create DataFrame
feature_names_fc = ([f'Intra_{n}' for n in network_names] + 
                    [f'Inter_{network_names[i]}_{network_names[j]}' 
                     for i in range(len(network_names)) for j in range(i+1, len(network_names))])
df_fmri = pd.DataFrame(fmri_features, index=feature_names_fc).T
df_fmri.index.name = 'Subject'
print(f"\nfMRI Feature Matrix: {df_fmri.shape}")
df_fmri.head()

### 3. QC: Visualize FC matrix for one subject

---

In [ ]:
# Plot FC matrix for one subject (same visualization as Lab 4)
example_sub = all_subjects[0]
work_dir = f'/content/fmri_work/{example_sub}'

masker = maskers.NiftiLabelsMasker(labels_img=atlas_filename, standardize='zscore_sample', verbose=0)
ts = masker.fit_transform(f'{work_dir}/func_filtered.nii.gz')
fc = connectome.ConnectivityMeasure(kind='correlation').fit_transform([ts])[0]

plt.figure(figsize=(8, 6))
plt.imshow(fc, interpolation='nearest', cmap='RdBu_r', vmin=-1, vmax=1)
plt.title(f'Functional Connectivity Matrix - {example_sub} (Schaefer 100)')
plt.colorbar(label='Pearson Correlation')
plt.xlabel('Brain Regions')
plt.ylabel('Brain Regions')
plt.tight_layout()
plt.show()

# **VI. EEG Features (Lab 5)**

We extract **spectral power** features from resting-state EEG using MNE-Python.

**Pipeline:** Load data → Preprocessing (filter + ICA, as in Lab 5) → PSD computation → Band power extraction

**Features extracted per subject:** Mean power in delta (1-4Hz), theta (4-8Hz), alpha (8-13Hz), beta (13-30Hz), and alpha peak frequency (APF) = 5 values.

---

### 1. EEG preprocessing

Apply the preprocessing pipeline from Lab 5: bandpass filter, ICA denoising with mne_icalabel.

---

In [ ]:
import mne
from mne_icalabel import label_components

eeg_features = {}

for sub in all_subjects:
    print(f"\n{'='*50}")
    print(f"Processing {sub} - EEG")
    print(f"{'='*50}")
    
    # Adapt path to your actual EEG file location and format
    # LEMON EEG is typically in BrainVision (.vhdr) or EEGLAB (.set) format
    eeg_file = f'/content/eeg_data/{sub}/eeg/{sub}_task-rest_eeg.vhdr'
    
    if not os.path.exists(eeg_file):
        print(f"  ⚠️ EEG file not found for {sub}")
        eeg_features[sub] = [np.nan] * 5
        continue
    
    try:
        # Load raw EEG
        raw = mne.io.read_raw_brainvision(eeg_file, preload=True)
        print(f"  Channels: {len(raw.ch_names)}, Sfreq: {raw.info['sfreq']} Hz")
        
        # Step 1: Bandpass filter (Lab 5)
        print("  Step 1: Bandpass filter (1-40 Hz)...")
        raw.filter(l_freq=1.0, h_freq=40.0)
        
        # Step 2: Set average reference
        raw.set_eeg_reference('average')
        
        # Step 3: ICA denoising (Lab 5)
        print("  Step 2: ICA denoising...")
        ica = mne.preprocessing.ICA(n_components=20, random_state=42)
        ica.fit(raw)
        
        # Classify components with ICLabel (Lab 5)
        ic_labels = label_components(raw, ica, method='iclabel')
        
        # Remove non-brain components
        exclude_idx = [i for i, label in enumerate(ic_labels['labels']) 
                       if label not in ('brain', 'other')]
        print(f"  Excluding {len(exclude_idx)} non-brain ICs: {exclude_idx}")
        ica.exclude = exclude_idx
        raw_clean = ica.apply(raw.copy())
        
        print(f"  ✓ Preprocessing done for {sub}")
        
        # Step 4: Compute PSD (Lab 5)
        print("  Step 3: Computing PSD...")
        psd = raw_clean.compute_psd(fmin=1.0, fmax=40.0, method='welch')
        freqs = psd.freqs
        psds = psd.get_data()  # (channels, freqs)
        
        # Average across channels
        mean_psd = np.mean(psds, axis=0)
        
        # Step 5: Extract band power
        bands = {
            'delta': (1, 4),
            'theta': (4, 8),
            'alpha': (8, 13),
            'beta': (13, 30)
        }
        
        band_powers = []
        for band_name, (fmin, fmax) in bands.items():
            band_mask = (freqs >= fmin) & (freqs <= fmax)
            band_power = np.mean(mean_psd[band_mask])
            band_powers.append(band_power)
        
        # Alpha Peak Frequency (APF)
        alpha_mask = (freqs >= 8) & (freqs <= 13)
        alpha_psd = mean_psd[alpha_mask]
        alpha_freqs = freqs[alpha_mask]
        apf = alpha_freqs[np.argmax(alpha_psd)]
        band_powers.append(apf)
        
        eeg_features[sub] = band_powers
        print(f"  ✓ Features: delta={band_powers[0]:.2e}, theta={band_powers[1]:.2e}, "
              f"alpha={band_powers[2]:.2e}, beta={band_powers[3]:.2e}, APF={apf:.1f} Hz")
        
    except Exception as e:
        print(f"  ⚠️ Error: {e}")
        eeg_features[sub] = [np.nan] * 5

# Create DataFrame
eeg_feature_names = ['Delta_Power', 'Theta_Power', 'Alpha_Power', 'Beta_Power', 'APF']
df_eeg = pd.DataFrame(eeg_features, index=eeg_feature_names).T
df_eeg.index.name = 'Subject'
print("\nEEG Feature Matrix:")
df_eeg

### 2. QC: Visualize PSD for one subject

---

In [ ]:
# Plot PSD for one subject (same style as Lab 5)
# This cell assumes raw_clean from the last processed subject is still in memory
# In practice, recompute for the subject you want to visualize

fig, ax = plt.subplots(figsize=(10, 4))
psd.plot(axes=ax, show=False)
ax.set_title(f'Power Spectral Density - {all_subjects[-1]}')
ax.axvspan(8, 13, alpha=0.2, color='orange', label='Alpha band (8-13 Hz)')
ax.legend()
plt.tight_layout()
plt.show()

# **VII. Feature Integration**

Combine all features from the 4 modalities into a single feature matrix per subject.

---

In [ ]:
# Combine all feature DataFrames
df_all = pd.concat([df_mri, df_dwi, df_fmri, df_eeg], axis=1)

# Add labels
df_all['Group'] = label_names
df_all['Label'] = labels

print(f"Combined feature matrix: {df_all.shape}")
print(f"  MRI features:  {df_mri.shape[1]}")
print(f"  DWI features:  {df_dwi.shape[1]}")
print(f"  fMRI features: {df_fmri.shape[1]}")
print(f"  EEG features:  {df_eeg.shape[1]}")
print(f"  Total features: {df_mri.shape[1] + df_dwi.shape[1] + df_fmri.shape[1] + df_eeg.shape[1]}")
print(f"  Subjects: {df_all.shape[0]}")

# Check for missing data
n_missing = df_all.drop(columns=['Group', 'Label']).isna().sum().sum()
print(f"\n  Missing values: {n_missing}")

df_all.head()

In [ ]:
# Visualize feature distributions by group
fig, axes = plt.subplots(2, 2, figsize=(14, 10))

# MRI: Example - L hippocampus volume
if 'L_Hipp' in df_all.columns:
    sns.boxplot(data=df_all, x='Group', y='L_Hipp', ax=axes[0, 0], palette='Set2')
    axes[0, 0].set_title('MRI: Left Hippocampus Volume')
    axes[0, 0].set_ylabel('Volume (mm³)')

# DWI: Example - Genu CC FA
if 'Genu_CC' in df_all.columns:
    sns.boxplot(data=df_all, x='Group', y='Genu_CC', ax=axes[0, 1], palette='Set2')
    axes[0, 1].set_title('DWI: Genu CC - FA')
    axes[0, 1].set_ylabel('FA')

# fMRI: Example - Intra-DMN connectivity
if 'Intra_Default' in df_all.columns:
    sns.boxplot(data=df_all, x='Group', y='Intra_Default', ax=axes[1, 0], palette='Set2')
    axes[1, 0].set_title('fMRI: Intra-DMN Connectivity')
    axes[1, 0].set_ylabel('Mean Correlation')

# EEG: Alpha Peak Frequency
if 'APF' in df_all.columns:
    sns.boxplot(data=df_all, x='Group', y='APF', ax=axes[1, 1], palette='Set2')
    axes[1, 1].set_title('EEG: Alpha Peak Frequency')
    axes[1, 1].set_ylabel('Frequency (Hz)')

plt.suptitle('Feature Distributions by Age Group (Example Features)', fontsize=14, fontweight='bold')
plt.tight_layout()
plt.show()

# **VIII. Classification: Young vs Old**

We train simple classifiers using Leave-One-Out Cross-Validation (LOOCV) and compare unimodal vs multimodal performance.

**Important:** With N=10, we expect noisy results. The goal is to demonstrate the process and discuss limitations, NOT to achieve high accuracy.

---

### 1. Prepare data

Handle missing values and normalize features.

---

In [ ]:
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer

# Separate features and labels
y = np.array(labels)

# Define feature sets per modality
feature_sets = {
    'MRI':  df_mri.values,
    'DWI':  df_dwi.values,
    'fMRI': df_fmri.values,
    'EEG':  df_eeg.values,
    'All':  df_all.drop(columns=['Group', 'Label']).values
}

# Impute missing values (mean) and standardize
processed_features = {}
for name, X in feature_sets.items():
    imputer = SimpleImputer(strategy='mean')
    X_imputed = imputer.fit_transform(X)
    scaler = StandardScaler()
    X_scaled = scaler.fit_transform(X_imputed)
    processed_features[name] = X_scaled
    print(f"{name:>5}: {X_scaled.shape[1]} features, {np.sum(np.isnan(X)):>3.0f} NaN imputed")

### 2. Leave-One-Out Classification

We use LOOCV because with N=10, k-fold would give very small test sets. Each iteration trains on 9 subjects and tests on 1.

---

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.model_selection import LeaveOneOut, cross_val_score, cross_val_predict
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

# Classifiers to try
classifiers = {
    'Logistic Regression': LogisticRegression(max_iter=1000, random_state=42),
    'SVM (linear)': SVC(kernel='linear', random_state=42),
}

loo = LeaveOneOut()

# Store results
results = []

for feat_name, X in processed_features.items():
    for clf_name, clf in classifiers.items():
        # LOOCV
        y_pred = cross_val_predict(clf, X, y, cv=loo)
        acc = accuracy_score(y, y_pred)
        
        results.append({
            'Modality': feat_name,
            'Classifier': clf_name,
            'Accuracy': acc,
            'Correct': int(acc * len(y)),
            'Total': len(y)
        })
        
        print(f"{feat_name:>5} + {clf_name:<25}: {acc:.1%} ({int(acc*len(y))}/{len(y)} correct)")

df_results = pd.DataFrame(results)

### 3. Visualize results

---

In [ ]:
# Plot classification accuracy by modality and classifier
fig, ax = plt.subplots(figsize=(10, 6))

pivot = df_results.pivot_table(index='Modality', columns='Classifier', values='Accuracy')
# Reorder modalities
order = ['MRI', 'DWI', 'fMRI', 'EEG', 'All']
pivot = pivot.reindex([m for m in order if m in pivot.index])

pivot.plot(kind='bar', ax=ax, colormap='Set2', edgecolor='black', width=0.7)

ax.set_ylim(0, 1.05)
ax.axhline(y=0.5, color='red', linestyle='--', linewidth=1, label='Chance level (50%)')
ax.set_ylabel('Accuracy (LOOCV)', fontsize=12)
ax.set_xlabel('Feature Set', fontsize=12)
ax.set_title('Classification Accuracy: Young vs Old\n(Leave-One-Out, N=10)', fontsize=14)
ax.legend(loc='lower right')
ax.set_xticklabels(ax.get_xticklabels(), rotation=0)

# Add value labels on bars
for container in ax.containers:
    ax.bar_label(container, fmt='%.0f%%', label_type='edge', fontsize=9,
                 padding=2, fontweight='bold')

plt.tight_layout()
plt.show()

In [ ]:
# Confusion matrix for the best multimodal model
best_row = df_results[df_results['Modality'] == 'All'].sort_values('Accuracy', ascending=False).iloc[0]
best_clf_name = best_row['Classifier']
best_clf = classifiers[best_clf_name]

y_pred_best = cross_val_predict(best_clf, processed_features['All'], y, cv=loo)
cm = confusion_matrix(y, y_pred_best)

fig, ax = plt.subplots(figsize=(5, 4))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=['Young', 'Old'],
            yticklabels=['Young', 'Old'], ax=ax, cbar=False, annot_kws={'size': 16})
ax.set_xlabel('Predicted', fontsize=12)
ax.set_ylabel('True', fontsize=12)
ax.set_title(f'Confusion Matrix - Multimodal ({best_clf_name})\nAccuracy: {best_row["Accuracy"]:.0%}', fontsize=13)
plt.tight_layout()
plt.show()

### 4. Feature importance analysis

Which features (and modalities) contribute most to classification?

---

In [ ]:
# Train a Logistic Regression on ALL features to get coefficients
from sklearn.linear_model import LogisticRegression

clf_final = LogisticRegression(max_iter=1000, random_state=42)
clf_final.fit(processed_features['All'], y)

# Get feature names
all_feature_names = (list(structures) + list(jhu_tracts.keys()) + 
                     feature_names_fc + eeg_feature_names)

# Get absolute coefficient values
coefs = np.abs(clf_final.coef_[0])

# Assign modality to each feature
modality_labels = (['MRI'] * len(structures) + ['DWI'] * len(jhu_tracts) + 
                   ['fMRI'] * len(feature_names_fc) + ['EEG'] * len(eeg_feature_names))

# Create importance DataFrame
df_importance = pd.DataFrame({
    'Feature': all_feature_names[:len(coefs)],
    'Importance': coefs,
    'Modality': modality_labels[:len(coefs)]
}).sort_values('Importance', ascending=False)

# Plot top 15 features
fig, ax = plt.subplots(figsize=(10, 6))
top15 = df_importance.head(15)
colors = {'MRI': '#3498DB', 'DWI': '#E74C3C', 'fMRI': '#2ECC71', 'EEG': '#F1C40F'}
bar_colors = [colors[m] for m in top15['Modality']]

bars = ax.barh(range(len(top15)), top15['Importance'].values, color=bar_colors, edgecolor='black')
ax.set_yticks(range(len(top15)))
ax.set_yticklabels(top15['Feature'].values, fontsize=10)
ax.invert_yaxis()
ax.set_xlabel('Absolute Coefficient', fontsize=12)
ax.set_title('Top 15 Most Important Features for Classification', fontsize=14)

# Legend
from matplotlib.patches import Patch
legend_elements = [Patch(facecolor=c, edgecolor='black', label=m) for m, c in colors.items()]
ax.legend(handles=legend_elements, loc='lower right', fontsize=11)

plt.tight_layout()
plt.show()

In [ ]:
# Summary: average importance by modality
modality_importance = df_importance.groupby('Modality')['Importance'].mean().sort_values(ascending=False)
print("Average feature importance by modality:")
print("=" * 40)
for mod, imp in modality_importance.items():
    bar = '█' * int(imp / modality_importance.max() * 20)
    print(f"  {mod:>5}: {imp:.4f}  {bar}")

# **IX. Discussion and Limitations**

---

### Questions to address in your discussion:

1. **Results interpretation:**
   - Which modality performed best individually? Does this match your expectations based on the neuroscience literature?
   - Did the multimodal approach improve over the best unimodal approach?
   - Which specific features were most discriminative?

2. **Limitations (be honest!):**
   - With N=10, how reliable are these accuracy estimates? What is the expected variance?
   - What is the risk of overfitting, especially with the multimodal feature set?
   - How does the LOOCV strategy help or hurt in this context?
   - What preprocessing choices could have affected the results?

3. **What would you do differently with more data?**
   - How many subjects would be needed for reliable classification?
   - What additional preprocessing or feature extraction methods would you apply?
   - Would you use different classifiers or feature selection methods?

4. **Process reflection:**
   - Which step in the pipeline was most challenging? Why?
   - Where did you have to make methodological decisions? How did you decide?
   - What quality control checks were most informative?

---

**Write your discussion below:**

In [ ]:
# Your discussion here (use markdown cells above or print statements)
print("TODO: Write your discussion addressing the questions above.")

# **X. Summary**

---

In [ ]:
# Final summary table
print("=" * 70)
print("PROJECT 10 - MULTIMODAL AGE CLASSIFICATION SUMMARY")
print("=" * 70)
print(f"\nDataset: LEMON (MPI-Leipzig Mind-Brain-Body)")
print(f"Subjects: {len(all_subjects)} ({len(young_subjects)} young, {len(old_subjects)} old)")
print(f"\nFeatures extracted:")
print(f"  MRI  (Lab 1): {df_mri.shape[1]:>3} features (subcortical volumes from FIRST)")
print(f"  DWI  (Lab 2): {df_dwi.shape[1]:>3} features (FA from JHU tracts)")
print(f"  fMRI (Lab 4): {df_fmri.shape[1]:>3} features (network-level FC from Schaefer)")
print(f"  EEG  (Lab 5): {df_eeg.shape[1]:>3} features (band power + APF)")
print(f"  Total:        {df_mri.shape[1] + df_dwi.shape[1] + df_fmri.shape[1] + df_eeg.shape[1]:>3} features")
print(f"\nBest results (LOOCV):")
for _, row in df_results.sort_values('Accuracy', ascending=False).head(3).iterrows():
    print(f"  {row['Modality']:>5} + {row['Classifier']:<25}: {row['Accuracy']:.0%}")